<h1 align=center><font size = 5>Artificial Neural Networks - Forward Propagation</font></h1>

Obviously, neural networks for real problems are composed of many hidden layers and many more nodes in each layer. So, we can't continue making predictions using this very inefficient approach of computing the weighted sum at each node and the activation of each node manually. 

In order to code an automatic way of making predictions, let's generalize our network. A general network would take $n$ inputs, would have many hidden layers, each hidden layer having $m$ nodes, and would have an output layer. Although the network is showing one hidden layer, but we will code the network to have many hidden layers. Similarly, although the network shows an output layer with one node, we will code the network to have more than one node in the output layer.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width=600px>

<a id="item2"></a>

## Initialize a Network

In [8]:
from random import seed
import numpy as np

In [2]:
n = 2 # number of inputs
num_hidden_layers = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer

Now that we defined the structure of the network, let's go ahead and inititailize the weights and the biases in the network to random numbers. In order to be able to initialize the weights and the biases to random numbers, we will need to import the **Numpy** library.

In [3]:
import numpy as np # import the Numpy library

num_nodes_previous = n # number of nodes in the previous layer

network = {} # initialize network an an empty dictionary

# loop through each layer and randomly initialize the weights and biases associated with each node
# notice how we are adding 1 to the number of hidden layers in order to include the output layer
for layer in range(num_hidden_layers + 1): 
    
    # determine name of layer
    if layer == num_hidden_layers:
        layer_name = 'output'
        num_nodes = num_nodes_output
    else:
        layer_name = 'layer_{}'.format(layer + 1)
        num_nodes = m[layer]
    
    # initialize weights and biases associated with each node in the current layer
    network[layer_name] = {}
    for node in range(num_nodes):
        node_name = 'node_{}'.format(node+1)
        network[layer_name][node_name] = {
            'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
            'bias': np.around(np.random.uniform(size=1), decimals=2),
        }
    
    num_nodes_previous = num_nodes
    
print(network) # print network

{'layer_1': {'node_1': {'weights': array([0.65, 0.26]), 'bias': array([0.82])}, 'node_2': {'weights': array([0.49, 0.82]), 'bias': array([0.69])}}, 'layer_2': {'node_1': {'weights': array([0.71, 0.16]), 'bias': array([0.47])}, 'node_2': {'weights': array([0.26, 0.47]), 'bias': array([0.5])}}, 'output': {'node_1': {'weights': array([0.95, 0.37]), 'bias': array([0.98])}}}


Awesome! So now with the above code, we are able to initialize the weights and the biases pertaining to any network of any number of hidden layers and number of nodes in each layer. But let's put this code in a function so that we are able to repetitively execute all this code whenever we want to construct a neural network.


In [5]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):
        
        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes

    return network # return the network

#### Use the *initialize_network* function to create a network that:

1. takes 5 inputs
2. has three hidden layers
3. has 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer
4. has 1 node in the output layer

Call the network **small_network**.

In [6]:
small_network = initialize_network(5, 5, [5, 4, 3, 4, 5], 1) # 5 input node, 5 hidden layers, 1 output node 

## Compute Weighted Sum at Each Node

The weighted sum at each node is computed as the dot product of the inputs and the weights plus the bias. So let's create a function called *compute_weighted_sum* that does just that.

In [7]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

Let's generate 5 inputs that we can feed to **small_network**.

In [9]:
np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


#### Use the *compute_weighted_sum* function to compute the weighted sum at the first node in the first hidden layer.

In [10]:
node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']

weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias)
print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))

The weighted sum at the first node in the hidden layer is 1.3387


## Compute Node Activation

Recall that the output of each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.

In [11]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

#### Use the *node_activation* function to compute the output of the first node in the first hidden layer.

In [13]:
node_output = node_activation(compute_weighted_sum(inputs, node_weights, node_bias))

print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))

The weighted sum at the first node in the hidden layer is 0.7923


## Forward Propagation

The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the *compute_weighted_sum* and *node_activation* functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.

The way we are going to accomplish this is through the following procedure:

1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
5. Repeat steps 2 - 4 until we compute the output of the output layer.

In [14]:
def forward_propagate(network, inputs):
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer]
        
        layer_outputs = [] 
        for layer_node in layer_data:
        
            node_data = layer_data[layer_node]
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

#### Use the *forward_propagate* function to compute the prediction of our small network

In [15]:
predictions = forward_propagate(small_network, inputs)

print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))

The outputs of the nodes in hidden layer number 1 is [0.7923, 0.8804, 0.7877, 0.7957, 0.8076]
The outputs of the nodes in hidden layer number 2 is [0.9202, 0.8909, 0.9528, 0.9062]
The outputs of the nodes in hidden layer number 3 is [0.8599, 0.9024, 0.8806]
The outputs of the nodes in hidden layer number 4 is [0.7239, 0.9368, 0.7291, 0.9188]
The outputs of the nodes in hidden layer number 5 is [0.9298, 0.8963, 0.8143, 0.9502, 0.9322]
The predicted value by the network for the given input is 0.9401


So we built the code to define a neural network. We can specify the number of inputs that a neural network can take, the number of hidden layers as well as the number of nodes in each hidden layer, and the number of nodes in the output layer.

We first use the *initialize_network* to create our neural network and define its weights and biases.

In [17]:
my_network = initialize_network(5, 5, [5, 40, 30, 40, 5], 7)

Then, for a given input,

In [18]:
inputs = np.around(np.random.uniform(size=5), decimals=2)

we compute the network predictions.

In [19]:
predictions = forward_propagate(my_network, inputs)
print('The predicted values by the network for the given input are {}'.format(predictions))

The outputs of the nodes in hidden layer number 1 is [0.8918, 0.9058, 0.9086, 0.8286, 0.9156]
The outputs of the nodes in hidden layer number 2 is [0.9367, 0.9731, 0.8952, 0.9113, 0.9702, 0.9359, 0.9645, 0.9612, 0.8968, 0.958, 0.9639, 0.974, 0.9703, 0.9362, 0.8732, 0.9219, 0.8413, 0.8306, 0.8707, 0.8486, 0.9608, 0.9565, 0.8765, 0.8569, 0.9484, 0.952, 0.8921, 0.9174, 0.8314, 0.924, 0.9879, 0.9648, 0.9333, 0.9145, 0.964, 0.9258, 0.9454, 0.932, 0.9306, 0.9716]
The outputs of the nodes in hidden layer number 3 is [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
The outputs of the nodes in hidden layer number 4 is [1.0, 1.0, 0.9999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
The outputs of the nodes in hidden layer number 5 is [1.0, 1.0, 1.0, 1.0, 1.0

Feel free to play around with the code by creating different networks of different structures and enjoy making predictions using the *forward_propagate* function.

In [21]:
my_network = initialize_network(4, 4, [4, 40, 30, 4], 70)
inputs = np.around(np.random.uniform(size=4), decimals=2)
predictions = forward_propagate(my_network, inputs)
print('The predicted values by the network for the given input are {}'.format(predictions))

The outputs of the nodes in hidden layer number 1 is [0.7545, 0.7974, 0.8686, 0.8269]
The outputs of the nodes in hidden layer number 2 is [0.8814, 0.8855, 0.8524, 0.857, 0.8774, 0.8814, 0.8049, 0.8416, 0.9022, 0.9108, 0.8697, 0.8779, 0.8893, 0.8792, 0.7586, 0.8098, 0.8464, 0.9187, 0.923, 0.9147, 0.8714, 0.8935, 0.9, 0.8352, 0.8472, 0.9049, 0.8513, 0.9238, 0.8819, 0.8966, 0.7184, 0.9084, 0.8406, 0.7947, 0.7805, 0.8355, 0.9386, 0.9056, 0.909, 0.9165]
The outputs of the nodes in hidden layer number 3 is [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
The outputs of the nodes in hidden layer number 4 is [1.0, 1.0, 1.0, 1.0]
The predicted values by the network for the given input are [0.9756, 0.922, 0.9011, 0.9565, 0.9234, 0.9589, 0.7631, 0.9105, 0.8744, 0.9321, 0.8557, 0.7291, 0.9047, 0.9601, 0.9707, 0.8085, 0.9597, 0.968, 0.9612, 0.9483, 0.9129, 0.8618, 0.8957, 0.9416, 0.9535, 0.9503, 0

In [23]:
my_network = initialize_network(10, 10, [10, 9, 8, 7, 6, 5, 4, 3, 2, 10], 1000)
inputs = np.around(np.random.uniform(size=10), decimals=2)
predictions = forward_propagate(my_network, inputs)
print('The predicted values by the network for the given input are {}'.format(predictions))

The outputs of the nodes in hidden layer number 1 is [0.9643, 0.9765, 0.9444, 0.9489, 0.9572, 0.9649, 0.9254, 0.9683, 0.9759, 0.9255]
The outputs of the nodes in hidden layer number 2 is [0.993, 0.9763, 0.9881, 0.9985, 0.9955, 0.9967, 0.9977, 0.9951, 0.9956]
The outputs of the nodes in hidden layer number 3 is [0.9915, 0.9908, 0.9938, 0.9909, 0.9904, 0.9988, 0.9808, 0.9686]
The outputs of the nodes in hidden layer number 4 is [0.987, 0.9489, 0.988, 0.994, 0.9887, 0.9954, 0.9875]
The outputs of the nodes in hidden layer number 5 is [0.9853, 0.9913, 0.9767, 0.9773, 0.9948, 0.9901]
The outputs of the nodes in hidden layer number 6 is [0.957, 0.9733, 0.9492, 0.9908, 0.959]
The outputs of the nodes in hidden layer number 7 is [0.8081, 0.9234, 0.9174, 0.9222]
The outputs of the nodes in hidden layer number 8 is [0.9613, 0.9454, 0.9007]
The outputs of the nodes in hidden layer number 9 is [0.9139, 0.8173]
The outputs of the nodes in hidden layer number 10 is [0.889, 0.7223, 0.8041, 0.8872, 0.